How would multi-agent work?

In [14]:
class Agent:
    def __init__(self, name, personality, full_prompt):
        self.name = name
        self.personality = personality
        self.full_prompt = full_prompt

    def respond(self, user_message=None, context=None):
        # ignore context for now
        if user_message is None:
            print(">", context)
            return "Well, that's interesting!"
        else:
            return "oh all right! You said: " + user_message
    
    def calling_card(self):
        return f"{self.name}, a {self.personality} agent."
    
    def __repr__(self):
        return f"Agent(name={self.name}, personality={self.personality}, full_prompt={self.full_prompt})"

a = Agent("Sally", "bright and cheerful", "You help the user find the best deals on groceries")
b = Agent("Bob", "sarcastic", "Pretend to be a user who is asking for help finding the best deals on groceries")

from pydantic import BaseModel
from typing import Any

class ConvoMessage(BaseModel):
    agent: str
    message: str
    
    def __repr__(self) -> str:
        return f"ConvoMessage(agent={self.agent}, message={self.message})"


class Convo:
    def __init__(self, agents):
        self.agents = agents
        self.messages = []
        self.current_agent_idx = 0
        self.turn_number = 0

    def _next(self):
        """Passes turn to the next agent"""
        self.current_agent_idx += 1
        self.current_agent_idx = self.current_agent_idx % len(self.agents)
        self.current_agent = self.agents[self.current_agent_idx]

    def describe_other_agents(self, agent):
        """Describes the other agents to the given agent"""
        return ", ".join([a.calling_card() for a in self.agents if a != agent])

    def _build_context(self, agent):
        """Builds the conversational context for the agent"""
        system_prompt = f"""
        You are {agent.name}.
        You are {agent.personality}.
        You are {agent.full_prompt}.
        """
        
        if self.turn_number == 0:
            context = system_prompt + f"\nYou are now entering a conversation with {self.describe_other_agents(agent)}\n"
        else:
            context = system_prompt + f"\nYou are in a conversation with {self.describe_other_agents(agent)}\n"
            for message in self.messages:
                context += message.model_dump_json() + "\n"
        return context

    def start(self, agent_to_go_first=None, turns=5):
        for i in range(turns):
            if self.turn_number == 0:
                self.current_agent = agent_to_go_first
            else:
                self.current_agent = self.agents[self.current_agent_idx]

            current_agent_name = self.current_agent.name
            
            if len(self.messages) == 0:
                message = self.current_agent.respond("Start the conversation")
            else:
                message = self.current_agent.respond(context=self._build_context(self.current_agent))
            self.messages.append(ConvoMessage(
                agent=current_agent_name,
                message=message
            ))

            # next agent
            self._next()
            self.turn_number += 1

    def __repr__(self):
        return f"Convo(agents={self.agents})"


c = Convo([a, b])
c.start(a)

print('\nThe final messages are:')
print(c.messages)












> 
        You are Bob.
        You are sarcastic.
        You are Pretend to be a user who is asking for help finding the best deals on groceries.
        
You are in a conversation with Sally, a bright and cheerful agent.
{"agent":"Sally","message":"oh all right! You said: Start the conversation"}

> 
        You are Sally.
        You are bright and cheerful.
        You are You help the user find the best deals on groceries.
        
You are in a conversation with Bob, a sarcastic agent.
{"agent":"Sally","message":"oh all right! You said: Start the conversation"}
{"agent":"Bob","message":"Well, that's interesting!"}

> 
        You are Bob.
        You are sarcastic.
        You are Pretend to be a user who is asking for help finding the best deals on groceries.
        
You are in a conversation with Sally, a bright and cheerful agent.
{"agent":"Sally","message":"oh all right! You said: Start the conversation"}
{"agent":"Bob","message":"Well, that's interesting!"}
{"agent":"Sally"